<a href="https://colab.research.google.com/github/obete/ClassifierDINOv3/blob/main/ClassifierDINOv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **DINOv3 as base with a classifier head**


In [ ]:
#@title Dependencies and Modules
!pip install tensorflow pillow requests

import os
import pandas as pd
import requests
from PIL import Image
from io import BytesIO


**Dataset Used**

The data used in this project is Fitzpartick17k dataset available in kaggle. Its a csv comprising of approximately 17,000 record of image url, fitzpatick scale and labels of skin condition for each image.

In [ ]:
#@title Setting Data Path
from google.colab import drive
drive.mount('/content/drive')
file_path = "/content/drive/My Drive/Colab Notebooks/AI_ML_EXAM/fitzpatrick17k.csv"

Mounted at /content/drive


In [ ]:
#@title Loading Dataset Google drive onto this notebook
skin_data = pd.read_csv(file_path)
skin_data.head()

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,url,url_alphanum
0,5e82a45bc5d78bd24ae9202d194423f8,3,3,drug induced pigmentary changes,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicmminoc...
1,fa2911a9b13b6f8af79cb700937cc14f,1,1,photodermatoses,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicpphoto...
2,d2bac3c9e4499032ca8e9b07c7d3bc40,2,3,dermatofibroma,benign dermal,benign,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicdderma...
3,0a94359e7eaacd7178e06b2823777789,1,1,psoriasis,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicppsori...
4,a39ec3b1f22c08a421fa20535e037bba,1,1,psoriasis,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicppsori...


**FIltering the Data**

This study is focusing on the Afican skin which falls under fitzpatrick scal 4, 5 and 6. There fore not all the 17,000 images in the data set will be useful for this project. We filtered out to remain with only desired scale.
This leave us with 4934 images corresponding to the desired skin color.

In [ ]:
#@title Filtering Data for Fitzpatrick Scale 5 and 6 and dropping the empty urls
filtered_data = skin_data[skin_data['fitzpatrick_scale'].isin([5,6]) & skin_data['url'].notna()]
filtered_data[['url','label','fitzpatrick_scale']].describe()

,fitzpatrick_scale
count,2158.000000
mean,5.291474
std,0.454546
min,5.000000
25%,5.000000
50%,5.000000
75%,6.000000
max,6.000000


In [ ]:
# Target folder under My Drive
DRIVE_ROOT = '/content/drive/MyDrive/Colab Notebooks/AI_ML_EXAM/'
base_dir = f'{DRIVE_ROOT}/skin_images'
os.makedirs(base_dir, exist_ok=True)

**Accessing Images.**

The .csv fil provides url to the public location of the images. Therefore, to make use of it, in this section, we download, resize  and save the images in their correct labels in our google drive in a folder skin_images.                                                                   This section does that and takes a count of the total number of images that have successfully been downloaded.

In [ ]:
#@title Downloading and Resizing Images
from pathlib import Path
# Target folder under My Drive
DRIVE_ROOT = '/content/drive/MyDrive/Colab Notebooks/AI_ML_EXAM/'
base_dir = f'{DRIVE_ROOT}/skin_images'
os.makedirs(base_dir, exist_ok=True)

# Remove duplicates from the dataset
filtered_data = filtered_data.drop_duplicates(subset=['url'])

# Headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
}

# Initialize counters
total_images_downloaded = 0
failed_urls = []

# Download, resize, and save images
for idx, row in filtered_data.iterrows():
    lbl, url = row['label'], row['url']
    outdir = os.path.join(base_dir, lbl)
    os.makedirs(outdir, exist_ok=True)
    try:
        # Use headers in the request
        resp = requests.get(url, headers=headers, timeout=10)  # Add headers
        resp.raise_for_status()

        # Process and save the image
        img = Image.open(BytesIO(resp.content)).convert('RGB')
        img = img.resize((224, 224))

        # Ensure unique filenames
        unique_filename = f"{idx}_{hash(url)}.jpg"
        img.save(os.path.join(outdir, unique_filename))
        total_images_downloaded += 1
    except Exception as e:
        failed_urls.append((url, str(e)))  # Log failed URLs

# Verify and count total files
file_count = sum(1 for _ in Path(base_dir).rglob('*.jpg'))

# Display results
print(f"Total images downloaded and stored: {total_images_downloaded}")
print(f"Total image files found in folder and subfolders: {file_count}")
print(f"Failed URLs: {len(failed_urls)}")

# Optional: Save failed URLs for review
failed_urls_path = f'{DRIVE_ROOT}/failed_urls.txt'
with open(failed_urls_path, 'w') as f:
    for url, error in failed_urls:
        f.write(f"{url}\t{error}\n")
print(f"Failed URLs logged to: {failed_urls_path}")



Total images downloaded and stored: 2155
Total image files found in folder and subfolders: 2155
Failed URLs: 3
Failed URLs logged to: /content/drive/MyDrive/Colab Notebooks/AI_ML_EXAM//failed_urls.txt


In [ ]:
import random
import shutil

for class_name in os.listdir(f'{DRIVE_ROOT}/skin_images'):
    class_path = os.path.join(f'{DRIVE_ROOT}/skin_images', class_name)
    if os.path.isdir(class_path):
        train_class_path = f"{DRIVE_ROOT}/train_set/{class_name}"
        val_class_path = f"{DRIVE_ROOT}/val_set/{class_name}"
        os.makedirs(train_class_path, exist_ok=True)
        os.makedirs(val_class_path, exist_ok=True)

        # List all images
        images = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
        random.shuffle(images)

        # Split 80/20
        split_idx = int(0.8 * len(images))
        train_files = images[:split_idx]
        val_files = images[split_idx:]

        # Copy into train/
        for f in train_files:
            shutil.copy(os.path.join(class_path, f), os.path.join(train_class_path, f))

        # Copy into val/
        for f in val_files:
            shutil.copy(os.path.join(class_path, f), os.path.join(val_class_path, f))

In [ ]:
from torchvision import transforms

# For ViT-Small/Base: 224x224, for ViT-Large: 384x384
img_size = 224

transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225))
])


In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

train_dir = f'{DRIVE_ROOT}/train_set'
val_dir = f'{DRIVE_ROOT}/val_set'

train_dataset = ImageFolder(root=train_dir, transform=transform)
val_dataset = ImageFolder(root=val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Class names
print(train_dataset.classes)


['acanthosis nigricans', 'acne', 'acne vulgaris', 'acquired autoimmune bullous diseaseherpes gestationis', 'acrodermatitis enteropathica', 'actinic keratosis', 'allergic contact dermatitis', 'aplasia cutis', 'basal cell carcinoma', 'becker nevus', 'behcets disease', 'calcinosis cutis', 'cheilitis', 'congenital nevus', 'dariers disease', 'dermatomyositis', 'disseminated actinic porokeratosis', 'drug eruption', 'drug induced pigmentary changes', 'dyshidrotic eczema', 'eczema', 'ehlers danlos syndrome', 'epidermal nevus', 'epidermolysis bullosa', 'erythema annulare centrifigum', 'erythema elevatum diutinum', 'erythema multiforme', 'erythema nodosum', 'factitial dermatitis', 'fixed eruptions', 'folliculitis', 'fordyce spots', 'granuloma annulare', 'granuloma pyogenic', 'hailey hailey disease', 'halo nevus', 'hidradenitis', 'ichthyosis vulgaris', 'incontinentia pigmenti', 'juvenile xanthogranuloma', 'kaposi sarcoma', 'keloid', 'keratosis pilaris', 'langerhans cell histiocytosis', 'lentigo m

In [ ]:
#@title Using DINOv3

!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 126.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


Local Inference on GPU
Model page: https://huggingface.co/facebook/dinov3-vith16plus-pretrain-lvd1689m

⚠️ If the generated code snippets do not work, please open an issue on either the model repo and/or on huggingface.js 🙏

The model you are trying to use is gated. Please make sure you have access to it by visiting the model page.To run inference, either set HF_TOKEN in your environment variables/ Secrets or run the following cell to login. 🤗

In [ ]:
from huggingface_hub import login
login(new_session=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# @title Load model directly
from transformers import AutoImageProcessor, AutoModel

processor = AutoImageProcessor.from_pretrained("facebook/dinov3-vith16plus-pretrain-lvd1689m")
backbone = AutoModel.from_pretrained("facebook/dinov3-vith16plus-pretrain-lvd1689m")

In [ ]:
#@title Freeze backbone → use DINOv3 as a fixed feature extractor.
for param in backbone.parameters():
    param.requires_grad = False  # freeze backbone


In [ ]:
#@title Custom Classification head
import torch.nn as nn

class DinoClassifier(nn.Module):
    def __init__(self, model, num_classes):
        super().__init__()
        self.model = model
        self.head = nn.Linear(model.config.hidden_size, num_classes)  # simple linear head

    def forward(self, x):
        features = self.model(x).last_hidden_state[:,0]  # CLS token
        return self.head(features)


In [ ]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 19.7 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"

model = DinoClassifier(backbone, num_classes=len(train_dataset.classes)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(5):  # Example: 5 epochs
    model.train()
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        outputs = model(imgs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 4.3372
Epoch 2, Loss: 3.7100
Epoch 3, Loss: 3.9488
Epoch 4, Loss: 3.6002
Epoch 5, Loss: 3.5411


In [ ]:
import torchmetrics

device = "cuda" if torch.cuda.is_available() else "cpu"

# Define metric objects
train_acc_metric = torchmetrics.Accuracy(task="multiclass", num_classes=len(train_dataset.classes)).to(device)
val_acc_metric   = torchmetrics.Accuracy(task="multiclass", num_classes=len(train_dataset.classes)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 10
for epoch in range(num_epochs):
    # ---- Training ----
    model.train()
    train_loss = 0
    train_acc_metric.reset()
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        outputs = model(imgs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * imgs.size(0)
        train_acc_metric.update(outputs, labels)

    train_loss /= len(train_dataset)
    train_acc = train_acc_metric.compute()

    # ---- Validation ----
    model.eval()
    val_loss = 0
    val_acc_metric.reset()
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * imgs.size(0)
            val_acc_metric.update(outputs, labels)

    val_loss /= len(val_dataset)
    val_acc = val_acc_metric.compute()

    print(f"Epoch {epoch+1}/{num_epochs} "
          f"| Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} "
          f"| Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")


Epoch 1/10 | Train Loss: 3.5902, Acc: 0.2018 | Val Loss: 3.7898, Acc: 0.1507
Epoch 2/10 | Train Loss: 3.4799, Acc: 0.2268 | Val Loss: 3.7230, Acc: 0.1720
Epoch 3/10 | Train Loss: 3.3899, Acc: 0.2423 | Val Loss: 3.6621, Acc: 0.1783
Epoch 4/10 | Train Loss: 3.3049, Acc: 0.2673 | Val Loss: 3.6037, Acc: 0.1996
Epoch 5/10 | Train Loss: 3.2244, Acc: 0.2786 | Val Loss: 3.5490, Acc: 0.1975
Epoch 6/10 | Train Loss: 3.1497, Acc: 0.2881 | Val Loss: 3.4985, Acc: 0.1975
Epoch 7/10 | Train Loss: 3.0765, Acc: 0.3000 | Val Loss: 3.4494, Acc: 0.2166
Epoch 8/10 | Train Loss: 3.0089, Acc: 0.3167 | Val Loss: 3.4048, Acc: 0.2187
Epoch 9/10 | Train Loss: 2.9467, Acc: 0.3304 | Val Loss: 3.3609, Acc: 0.2335
Epoch 10/10 | Train Loss: 2.8833, Acc: 0.3405 | Val Loss: 3.3214, Acc: 0.2378
